In [2]:
#training data
#conv2d -> relu -> max_pooling_2x2 -> conv2d -> relu -> max_pooling_2x2 -> full connected(matual + relu) ->dropout -> output layer

#convolution: 將整張圖片利用不同的特徵，去尋找每個位置不同的特徵強度
#pooling: 將圖片壓縮並保留重要資訊，放在convolution後可以有效利用每個點的特徵強度


import tensorflow as tf

def train_mnist():
    from tensorflow.examples.tutorials.mnist import input_data
    #將dataset讀到MNIT_DATA的資料夾
    #one_hot: only 0 or 1
    mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

    x = tf.placeholder(tf.float32, [None, 784])
    y_ = tf.placeholder(tf.float32, [None, 10])
    

    #整張圖的每個點在0~9的weight
    W = tf.Variable(tf.zeros([784, 10]))
    b = tf.Variable(tf.zeros([10]))


    #define function

    # weight_variable: -0.2~0.2
    #將shape的每個element設為從常態分佈隨機取一個數, 大於2個標準差的數會drop, re-pick
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)
    #bias = 0.1
    #Creates a constant tensor and initialize it to 0.1
    def bias_variable(shape):
        initial = tf.constant(0.1, shape = shape)
        return tf.Variable(initial)

    #2-D CNN
    #stride(位移量) = [1, stride,stride, 1]
    def conv2d(x, W):
        return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding = 'SAME')

    #x(input) = [batch, height, width, channels]
    #ksize(窗口大小) = [1, height, width, 1]
    #stride = [1, stride,stride, 1]
    def max_pool_2x2(x):
        return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding = 'SAME')



    #define first convolution
    #kernel: 5*5, input: 1(黑白), output: 32 featureMap
    W_conv1 = weight_variable([5, 5, 1, 32]) 
    b_conv1 = bias_variable([32])
    #input dimantion: -1(不考慮維度), size: 28*28, channel:1
    x_image = tf.reshape(x, [-1, 28, 28, 1]) 

    #first convolution
    #激活函式Relu
    #padding = SAME
    #output image: 28*28*32 (32features)
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

    #pooling
    #strides = 2*2
    #output image: 14*14*32
    h_pool1 = max_pool_2x2(h_conv1)



    #define second convolution 
    #kernel: 5*5
    #input: 32, output: 64
    W_conv2 = weight_variable([5, 5, 32, 64]) 
    b_conv2 = bias_variable([64])

    #2nd convolution
    #output: 14*14*64
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    #pooling
    #output: 7*7*64
    h_pool2 = max_pool_2x2(h_conv2)


    #full connected layer
    #flatten並接回原本 network
    #input: 7*7*64, output: 1024
    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])
    #flatten h_pool2: 3D -> 1D
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])

    #NOT CNN!!
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


    #dropout
    #避免過度擬合(over fitting)
    #keep_prob: run時使用, 1.0=100%
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


    #output layer
    #input: 1024, output: 10
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])


    #softmax
    #轉為0~1的機率
    y_predict = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2)+b_fc2)


    
    #cross entropy
    #衡量不同策略所需的成本
    #define cost: 找出logits and labels 之間的cross entropy
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_predict, labels=y_))
    #優化器: 使cross entropy最小化
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)


    #預測準確度
    #argmax: 得到Tensor中最大值的index
    correct_prediction = tf.equal(tf.argmax(y_predict,1), tf.argmax(y_,1))
    #神經網路準確率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


    #initialize
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())


    #train for 1000 times
    train_accuracy = 0.0
    for i in range(10):
        batch = mnist.train.next_batch(100)
        #跑訓練集，利用optimizer將結果導到較好的地方
        train_step.run(session=sess, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.7})
        if i%100 == 0:
            #輸出準確度
            train_accuracy = accuracy.eval(session=sess, feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
            print("step {}, training accuracy {:}".format(i, train_accuracy))
    
    print(sess.run(tf.argmax(correct_prediction, 1), feed_dict={load_model.img_placeholder: mnist.test.images}))
    return


train_mnist()



Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

step 0, training accuracy 0.07999999821186066


TypeError: Value passed to parameter 'input' has DataType bool not in list of allowed values: float32, float64, int32, uint8, int16, int8, complex64, int64, qint8, quint8, qint32, bfloat16, uint16, complex128, float16, uint32, uint64

In [2]:
## Save trained mnist weight to file:"my_net/save_net.ckpt"

# remember to define the same dtype and shape when restore

import tensorflow as tf
import numpy as np


# Create some variables.
W = tf.Variable([[1,2,3],[5,4,3]], dtype=tf.float32, name='weights')
b = tf.Variable([[1,2,3]], dtype=tf.float32, name='biases')




a = tf.Variable([[4,5,6]], dtype=tf.float32, name='a')
# Add an op to initialize the variables.
init = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, initialize the variables, do some work, and save the
# variables to disk.
sess = tf.Session()
sess.run(init)
save_path = saver.save(sess, "my_net/save_net.ckpt")
print("Model saved in path: ", save_path)



Model saved in path:  my_net/save_net.ckpt


In [2]:
#load file
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

# Create some variables.
W = tf.Variable(np.arange(6).reshape((2, 3)), dtype=tf.float32, name="weights")
b = tf.Variable(np.arange(3).reshape((1, 3)), dtype=tf.float32, name="biases")
a = tf.Variable(np.arange(3).reshape((1, 3)), dtype=tf.float32, name='a')
# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
    
    # Restore variables from disk.
    saver.restore(sess, "my_net/save_net.ckpt")
    print("Model restored.")
    # Check the values of the variables
    print("weights:", sess.run(W)) #W.eval()
    print("biases:", sess.run(a))
    


INFO:tensorflow:Restoring parameters from my_net/save_net.ckpt
Model restored.
weights: [[1. 2. 3.]
 [5. 4. 3.]]
biases: [[4. 5. 6.]]


In [25]:
#test CNN
#batch 100 images a round

test_accuracy = 0.0
BATCH_SIZE = 100
test_image = mnist.test.images
test_label = mnist.test.labels
batched_test_image = []
batched_test_label = []


for i in range(int(len(test_label) / BATCH_SIZE)):
    batched_test_image = []
    batched_test_label = []
    for _ in range(BATCH_SIZE):
        batched_test_image.append(test_image[i])
        batched_test_label.append(test_label[i])
    test_accuracy += accuracy.eval(session=sess, feed_dict={x: batched_test_image, y_: batched_test_label, keep_prob: 1.0})
    print(test_accuracy)

test_accuracy /= int(len(test_label) / BATCH_SIZE)
print(test_accuracy)

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

In [6]:
#print("test accuracy %g"%accuracy.eval(session=sess, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))





import numpy
import random
r = random.randint(0, len(mnist.test.images))
#print("test accuracy %g"%accuracy.eval(session=sess, feed_dict={x: mnist.test.images[r], y_: mnist.test.labels[r], keep_prob: 1.0}))


from PIL import Image

#open the images and convert to greyscale
img1 = Image.open("test_images/9.png").convert("L")
img_array = numpy.array(img1)
print(img_array)



test_accuracy = 0.0
BATCH_SIZE = 500
test_image = mnist.test.images
test_label = mnist.test.labels
batched_test_image = []
batched_test_label = []

for i in range(int(len(test_label) / BATCH_SIZE)):
    batched_test_image = []
    batched_test_label = []
    for _ in range(BATCH_SIZE):
        batched_test_image.append(test_image[i])
        batched_test_label.append(test_label[i])
    test_accuracy += accuracy.eval(session=sess, feed_dict={x: batched_test_image, y_: batched_test_label, keep_prob: 1.0})
    print(test_accuracy)

test_accuracy /= int(len(test_label) / BATCH_SIZE)
print(test_accuracy)



''' 

for i in range(len(test_label)):
    
    if(i % 100 == 0):
        test_accuracy += accuracy.eval(session=sess, feed_dict={x: batch_test_img[i+BATCH_SIZE], y_: batch_test_label[i+BATCH_SIZE], keep_prob: 1.0})
        print(test_accuracy)
     
test_accuracy /= (TESTDATA_SIZE / 5)
print("test accuracy {}".format(test_accuracy))
'''


FileNotFoundError: [Errno 2] No such file or directory: 'data\\train-labels-idx1-ubyte'